In [1]:
import pandas as pd
import numpy as np

In [2]:
from pycaret.classification import *

In [3]:
train = pd.read_csv('train_merged.csv')

In [4]:
train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLIENT_INSTALLMENT_AMT_PAYMENT_MIN_COUNT,CLIENT_INSTALLMENT_AMT_PAYMENT_MIN_MEAN,CLIENT_INSTALLMENT_AMT_PAYMENT_MIN_MAX,CLIENT_INSTALLMENT_AMT_PAYMENT_MIN_MIN,CLIENT_INSTALLMENT_AMT_PAYMENT_MIN_SUM,CLIENT_INSTALLMENT_AMT_PAYMENT_SUM_COUNT,CLIENT_INSTALLMENT_AMT_PAYMENT_SUM_MEAN,CLIENT_INSTALLMENT_AMT_PAYMENT_SUM_MAX,CLIENT_INSTALLMENT_AMT_PAYMENT_SUM_MIN,CLIENT_INSTALLMENT_AMT_PAYMENT_SUM_SUM
0,100002,True,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,19.0,9251.7750,9251.775,9251.775,175783.73,19.0,219625.700,219625.700,219625.700,4.172888e+06
1,100003,False,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,25.0,46164.3320,98356.990,6662.970,1154108.20,25.0,453952.220,1150977.400,80773.380,1.134881e+07
2,100004,False,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,3.0,5357.2500,5357.250,5357.250,16071.75,3.0,21288.465,21288.465,21288.465,6.386539e+04
3,100006,False,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,16.0,62154.7930,691786.900,2482.920,994476.70,16.0,232499.700,691786.900,25091.324,3.719995e+06
4,100007,False,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,66.0,7329.6420,16046.100,0.180,483756.38,66.0,172669.890,280199.700,18330.390,1.139621e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,False,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,7.0,6605.9100,6605.910,6605.910,46241.37,7.0,52450.470,52450.470,52450.470,3.671533e+05
307507,456252,False,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,6.0,10046.8800,10046.880,10046.880,60281.28,6.0,60419.203,60419.203,60419.203,3.625152e+05
307508,456253,False,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,14.0,2986.1420,5567.715,27.270,41805.99,14.0,21778.004,33413.758,8317.890,3.048920e+05
307509,456254,True,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,19.0,10239.8320,19065.824,2296.440,194556.81,19.0,93367.150,171592.420,22964.398,1.773976e+06


In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [7]:
cl = setup(data = train, target = 'TARGET',  remove_multicollinearity = True, multicollinearity_threshold = 0.9, remove_perfect_collinearity = True)

IntProgress(value=0, description='Processing: ', max=13)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:44:05
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
ETC,. . . . . . . . . . . . . . . . . .,Calculating ETC


MemoryError: Unable to allocate 3.64 GiB for an array with shape (1587, 307511) and data type float64

In [5]:
cl = setup(data = train, target = 'TARGET',  remove_multicollinearity = True, multicollinearity_threshold = 0.9, remove_perfect_collinearity = True, 
           polynomial_features = True, polynomial_degree = 2, 
           trigonometry_features = True, polynomial_threshold = 0.1,  
           feature_selection = True, feature_selection_threshold = 0.8, feature_interaction = True, 
           feature_ratio = True, interaction_threshold = 0.01, fix_imbalance = True, fix_imbalance_method = None, 
           data_split_shuffle = True, folds_shuffle = False, n_jobs = -1, html = True, session_id = None, 
           log_experiment = False, experiment_name = None, log_plots = False, log_profile = False, log_data = False, 
           silent=False, verbose=True, profile = False )

IntProgress(value=0, description='Processing: ', max=13)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:44:05
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
ETC,. . . . . . . . . . . . . . . . . .,Calculating ETC


Text(value="Following data types have been inferred automatically, if they are correct press enter to continue…

,Data Type
SK_ID_CURR,Numeric
TARGET,Label
NAME_CONTRACT_TYPE,Categorical
CODE_GENDER,Categorical
FLAG_OWN_CAR,Categorical
...,...
CLIENT_INSTALLMENT_AMT_PAYMENT_SUM_COUNT,Numeric
CLIENT_INSTALLMENT_AMT_PAYMENT_SUM_MEAN,Numeric
CLIENT_INSTALLMENT_AMT_PAYMENT_SUM_MAX,Numeric
CLIENT_INSTALLMENT_AMT_PAYMENT_SUM_MIN,Numeric


MemoryError: Unable to allocate 3.65 GiB for an array with shape (1592, 307511) and data type float64

In [ ]:
compare_models(blacklist = None, whitelist = ['ada', 'gbc','xgboost', 'lightgbm', 'catboost', 'lda'], fold = 10,  round = 4,  sort = ‘AUC’, n_select = 1, turbo = True, verbose = True)

In [ ]:
ETR = create_model('et')

In [ ]:
ET = tune_model(ETR, optimize='RMSE')

In [ ]:
test = pd.read_csv('../input/melanoma-tumor-size-prediction-machinehack/Test.csv')

In [ ]:
pred = predict_model(ET, data = test)

In [ ]:
submission = pd.DataFrame(pred, columns=['tumor_size'])
submission.to_csv('submission.csv')